In [4]:
import numpy as np
import matplotlib.pyplot as plt

import gmplot
import torch
from torch import nn
from tqdm import trange
from sklearn.preprocessing import  OneHotEncoder
import dataLoader
import torchTrain

In [5]:
# %run torchTrain.ipynb
# 上面这种引用ipynb的格式就不能带前面类的名字







In [6]:
def draw_two_old(list,number,b,k):
    new_list = torch.tensor(list)
    lat = []
    lng = []
    for i in new_list:
        lat.append(i[0])
        lng.append(i[1])
    old_lat = torch.stack(lat[:k-1])
    old_lng =torch.stack(lng[:k-1])
    print(old_lat)
    print(old_lng)
    lat_predict = (lat[k-1:])
    lng_predict = (lng[k-1:])
    print(lat_predict)
    print(lng_predict)
    lat = torch.stack(lat)
    lng =torch.stack(lng)
    gmap = gmplot.GoogleMapPlotter(lat[0], lng[0], b)
    gmap.plot(lat, lng,color='b',lw=20)  #描绘轨迹点
    gmap.plot(lat_predict, lng_predict,color='r',lw=20)  #描绘轨迹点
    gmap.draw("user{}.html".format(number))   #显示图
    print("over")


def draw(list,number):
    new_list = torch.tensor(list)
    lat = []
    lng = []
    for i in new_list:
        lat.append(i[0])
        lng.append(i[1])
    lat = torch.stack(lat)#拼接出来的环境是多了一层结构
    lng =torch.stack(lng)
    gmap = gmplot.GoogleMapPlotter(lat[0], lng[0], 20)
    gmap.plot(lat, lng,color='b',lw=20)  #描绘轨迹点
    # gmap.plot(lat_predict, lng_predict,color='r',lw=20)  #描绘轨迹点
    gmap.draw("user{}.html".format(number))   #显示图
    print("over")

def draw_two_new(list_pre,list_real,promot):
    lat_pre, lng_pre = change_list_to_lat_lng(list_pre)
    lat_real, lng_real = change_list_to_lat_lng(list_real)
    gmap = gmplot.GoogleMapPlotter(lat_pre[0], lng_pre[0], 20)# 前面2个参数定义了center
    gmap.plot(lat_pre, lng_pre,color='b',lw=30)  #描绘轨迹点
    gmap.plot(lat_real, lng_real,color='r',lw=30)  #描绘轨迹点
    gmap.draw("user{}.html".format(promot))   #显示图
    print("over")

def change_list_to_lat_lng(list):
    new_list = torch.tensor(list)
    lat = []
    lng = []
    for i in new_list:
        lat.append(i[0])
        lng.append(i[1])
    lat = torch.stack(lat)#拼接出来的环境是多了一层结构
    lng =torch.stack(lng)
    return lat, lng
# 搞一个批量训练的模型

In [7]:


# coincide_number= 0
# coincide = np.zeros(Train_h3_list.size)
#


train_dataloader,test_dataloader,vocab, Train_h3_list, Test_h3_list  = dataLoader.DataLoader.load_h3_list()

encoder = OneHotEncoder(sparse=False).fit(vocab.reshape(-1, 1))

#这个函数现在没有用上
def encoding(data):
    return encoder.transform(data.reshape(-1,1))
def decoding(one_hot_data):
    return encoder.inverse_transform(one_hot_data)


# 可以通过idnex找到对应的h3
index_h  = dict(enumerate(vocab))

# 可以通过h3编码找到对应的index
h_index ={h3:i for i ,h3 in index_h.items()}


def label_encode(data):
    return np.array([h_index[ch] for ch in data])
def label_decode(data):
    return np.array([index_h[ch] for ch in data])




geo_level : 8
Train_h3_list SIZE : 22025
Test_h3_list SIZE : 9327
the train_data appear in test_data : 11737
the test_data appear in train_data : 6716
vocab_size : 590


In [8]:



# def test_dataloader(y,k):
# 现在相当于是在往前看10步

#暂时不加批量训练
# def data_loader(dataset,batchsize,drop_last=True):
#     for i in range(0,len(dataset)-batchsize+1,batchsize):
#         batch = dataset[i:i+batchsize]
#         batch

In [9]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
class RNN(nn.Module):
    def __init__(self,vocab) -> None:
        super().__init__()
        self.vocab = vocab
        self.num_hiddens = 255
        self.model =nn.LSTM(
            input_size=vocab,
            hidden_size=self.num_hiddens,
            batch_first=True,
            bias=True
        )

        self.output = nn.Sequential(
            nn.Linear(self.num_hiddens,vocab)
        )


    def forward(self,x , state):
        r_out, states= self.model(x.view(1,10,self.vocab) ,state)
        outdata = self.output(r_out[:,-1,:])
        return outdata, states

    def begin_state(self, device, batch_size=1):
        return (torch.zeros((
            self.model.num_layers,
            batch_size, self.num_hiddens), device=device),
                torch.zeros((
                    self.model.num_layers,
                    batch_size, self.num_hiddens), device=device))

net = RNN(len(vocab)).to(device)
# net=nn.DataParallel(net) #这个暂时先别用
optimizer = torch.optim.SGD(net.parameters(),lr = 0.1,momentum=0.8)
optimizer_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss_function = nn.CrossEntropyLoss()#输入是没有softmax的，这个函数是自己softmax

In [ ]:


top_k = 5

epoches = 5



train_acc = []
train_loss = []
test_acc = []


for epoch in trange(epoches):


    optimizer_scheduler.step()
    optimizer.step()

    how_many_instance, loss_, how_many_instance_right = torchTrain.train_epoch(net,train_dataloader,loss_function,optimizer,device,top_k,label_decode,label_encode,encoder)

    train_loss.append(loss_.cpu().detach().numpy() / how_many_instance)
    train_acc.append(how_many_instance_right / how_many_instance)
    print(f'--------------Epochs{epoches} | {epoch}---------------')
    print(f'average Train Loss : {train_loss[-1]} , train acc : {train_acc[-1]}')

    # if epoch%5 == 0:
    how_many_instance, how_many_instance_right, predict_answer_list, real_answer_list = torchTrain.predict_epoch(net,test_dataloader,device,top_k,label_decode,encoder)

    test_acc.append(how_many_instance_right / how_many_instance )
    print(f'test accuracy : { test_acc[-1]} ')


plt.plot(train_acc, label='Average Train Accuracy')
plt.plot(train_loss, label='Average Train Loss')
plt.plot(test_acc, label='test_acc')
plt.title('Loss vs Epochs')
plt.legend()
plt.show()






  0%|          | 0/60 [00:00<?, ?it/s]C:\Users\Administrator\.conda\envs\AISUStech\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


--------------Epochs60 | 0---------------
average Train Loss : 1.733773315069271 , train acc : 0.7069725187372247


  2%|▏         | 1/60 [05:48<5:42:38, 348.45s/it]

test accuracy : 0.39003971235376195 
--------------Epochs60 | 1---------------
average Train Loss : 1.2793572919032479 , train acc : 0.768703156938451


  3%|▎         | 2/60 [10:49<5:10:05, 320.78s/it]

...

test accuracy : 0.01652892561983471 
--------------Epochs60 | 15---------------
average Train Loss : 0.5735480868867817 , train acc : 0.8841244605950488


 27%|██▋       | 16/60 [1:23:57<4:06:44, 336.46s/it]

test accuracy : 0.02329075882794891 
--------------Epochs60 | 16---------------
average Train Loss : 0.545814998296616 , train acc : 0.8837610719963661


 28%|██▊       | 17/60 [1:30:19<4:10:45, 349.90s/it]

test accuracy : 0.02919394654931845 


In [ ]:
# top_k = 5
# name_id = 1
# how_many_instance, how_many_instance_right, predict_answer_list = torchTrain.predict_epoch(net,test_dataloader,device,top_k,label_decode,encoder)
# predict_answer_geo_list = dataLoader.h3_t_geo(predict_answer_list)
# # old=h3_t_geo(i.tolist())
# # total_list = old+predict
# # print(total_list)
# draw(predict_answer_geo_list,name_id,20,10)
# name_id+=1
#
# print(predict_answer_list)
#
# torch.save(net,"model_2022_4_8.pt")
top_k = 5
name_id = 1

# how_many_instance, how_many_instance_right, predict_answer_list, real_answer_list = torchTrain.predict_epoch(net,test_dataloader,device,top_k,label_decode,encoder)
how_many_instance, how_many_instance_right, predict_answer_list, real_answer_list = torchTrain.predict_epoch(net,test_dataloader,device,top_k,label_decode,encoder)
# print(predict_answer_list)
predict_answer_geo_list = dataLoader.h3_t_geo(predict_answer_list)
real_answer_geo_list = dataLoader.h3_t_geo(real_answer_list)
# old=h3_t_geo(i.tolist())
# total_list = old+predict
# print(total_list)
# draw(predict_answer_geo_list,1)
# draw(real_answer_geo_list,2)
draw_two_new(predict_answer_geo_list,real_answer_geo_list,"prepare")
name_id+=1
